In [21]:
import json
from concurrent.futures import ThreadPoolExecutor

import boto3
from botocore import UNSIGNED
from botocore.config import Config
import pandas as pd
from web3 import Web3

from mainnet_launch.database.schema.full import RebalancePlans, Destinations, DexSwapSteps, Tokens

from mainnet_launch.database.schema.postgres_operations import (
    get_full_table_as_orm,
    get_full_table_as_df,
    insert_avoid_conflicts,
    get_subset_not_already_in_column,
)

from mainnet_launch.constants import ALL_AUTOPOOLS, AutopoolConstants, USDC, WETH, AUTO_ETH


autopool = AUTO_ETH

s3_client = boto3.client("s3", config=Config(signature_version=UNSIGNED))
rebalance_plan_df: list[RebalancePlans] = get_full_table_as_df(
    RebalancePlans, where_clause=(RebalancePlans.autopool_vault_address == autopool.autopool_eth_addr)
)
rebalance_plan_df

2025-05-28 10:21:30,626 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-28 10:21:30,627 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname_1)s
2025-05-28 10:21:30,627 INFO sqlalchemy.engine.Engine [cached since 275.9s ago] {'table_name': <sqlalchemy.sql.elements.TextClause object at 0x12c2ecee0>, 'param_1': 'r', 'param_2': 'p', 'param_3': 'f', 'param_4': 'v', 'param_5': 'm', 'nspname_1': 'pg_catalog'}
2025-05-28 10:21:30,627 INFO sqlalchemy.engine.Engine 
            SELECT *
            FROM rebalance_plans
            WHERE rebalance_plans.autopool_vault_address = '0x0A2b94F687

,file_name,datetime_generated,autopool_vault_address,chain_id,solver_address,rebalance_type,destination_out,token_out,destination_in,token_in,...,in_dest_apr,in_dest_adj_apr,apr_delta,swap_offset_period,num_candidate_destinations,candidate_destinations_rank,projected_swap_cost,projected_net_gain,projected_gross_gain,projected_slippage
0,rebalance_plan_1727897421_0x0A2b94F6871c1D7A32...,2024-10-02 19:30:21+00:00,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,1,0x952D7a7eB2e0804d37d9244BE8e47341356d2f5D,FromIdle,0x896eCc16Ab4AFfF6cE0765A5B924BaECd7Fa455a,0xe080027Bd47353b5D1639772b4a75E9Ed3658A0d,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,...,0.000000,0.092285,0.092285,50.0,12.0,0.0,0.198901,1.003935,1.202836,0.199381
1,rebalance_plan_1736580621_0x0A2b94F6871c1D7A32...,2025-01-11 07:30:21+00:00,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,1,0x952D7a7eB2e0804d37d9244BE8e47341356d2f5D,FromIdle,0xc4Eb861e7b66f593482a3D7E8adc314f6eEDA30B,0x88794C65550DeB6b4087B7552eCf295113794410,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,...,0.000000,0.086773,0.086773,46.0,5.0,0.0,0.056907,1.388123,1.445030,0.041353
2,rebalance_plan_1739302221_0x0A2b94F6871c1D7A32...,2025-02-11 19:30:21+00:00,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,1,0x952D7a7eB2e0804d37d9244BE8e47341356d2f5D,FromIdle,0xbA1462f43c6f60ebD1C62735c94E428aD073E01A,0xC8Eb2Cf2f792F77AF0Cd9e203305a585E588179D,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,...,0.000000,0.089842,0.089842,42.0,4.0,0.0,0.025624,0.408126,0.433750,0.059284
3,rebalance_plan_1737682221_0x0A2b94F6871c1D7A32...,2025-01-24 01:30:21+00:00,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,1,None,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,rebalance_plan_1743481821_0x0A2b94F6871c1D7A32...,2025-04-01 04:30:21+00:00,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,1,None,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
732,rebalance_plan_1748028624_0x0A2b94F6871c1D7A32...,2025-05-23 19:30:24+00:00,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,1,0x952D7a7eB2e0804d37d9244BE8e47341356d2f5D,FromIdle,0xbA1462f43c6f60ebD1C62735c94E428aD073E01A,0xC8Eb2Cf2f792F77AF0Cd9e203305a585E588179D,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,...,0.000000,0.040399,0.040399,40.0,3.0,0.0,-0.010328,0.050955,0.040627,-0.075315
733,rebalance_plan_1748255424_0x0A2b94F6871c1D7A32...,2025-05-26 10:30:24+00:00,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,1,0x952D7a7eB2e0804d37d9244BE8e47341356d2f5D,Pool2Pool,0xbA1462f43c6f60ebD1C62735c94E428aD073E01A,0xC8Eb2Cf2f792F77AF0Cd9e203305a585E588179D,0xc4Eb861e7b66f593482a3D7E8adc314f6eEDA30B,0x88794C65550DeB6b4087B7552eCf295113794410,...,0.028550,0.045355,0.016805,38.0,3.0,0.0,0.005870,0.047047,0.052917,0.015343
734,rebalance_plan_1748050224_0x0A2b94F6871c1D7A32...,2025-05-24 01:30:24+00:00,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,1,0x952D7a7eB2e0804d37d9244BE8e47341356d2f5D,FromIdle,0xe4433D00Cf48BFE0C672d9949F2cd2c008bffC04,0x6951bDC4734b9f7F3E1B74afeBC670c736A0EDB6,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,...,0.000000,0.043558,0.043558,40.0,4.0,0.0,-0.005651,0.060400,0.054749,-0.041204
735,rebalance_plan_1748082624_0x0A2b94F6871c1D7A32...,2025-05-24 10:30:24+00:00,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,1,0x952D7a7eB2e0804d37d9244BE8e47341356d2f5D,Pool2Pool,0xbA1462f43c6f60ebD1C62735c94E428aD073E01A,0xC8Eb2Cf2f792F77AF0Cd9e203305a585E588179D,0xc4Eb861e7b66f593482a3D7E8adc314f6eEDA30B,0x88794C65550DeB6b4087B7552eCf295113794410,...,0.030014,0.048076,0.018063,40.0,3.0,0.0,0.018169,0.129081,0.147250,0.018439


In [22]:
from mainnet_launch.data_fetching.tokemak_subgraph import fetch_autopool_rebalance_events_from_subgraph

rebalance_event_df = fetch_autopool_rebalance_events_from_subgraph(autopool)
rebalance_event_df

,transactionHash,timestamp,blockNumber,tokenInAmount,tokenIn,tokenOutAmount,tokenOut,destinationInAddress,destinationOutAddress,tokenInAddress,tokenOutAddress
20,0xf4b0efb2e5d69ef7b7d000b4860a9d86b76013d9a042...,1726602143,20772483,99.961014,{'id': '0xc8eb2cf2f792f77af0cd9e203305a585e588...,100.000000,{'id': '0xc02aaa39b223fe8d0a0e5c4f27ead9083c75...,0xE382BBd32C4E202185762eA433278f4ED9E6151E,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,0xC8Eb2Cf2f792F77AF0Cd9e203305a585E588179D,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2
6,0xfdb95550d0720ff1f6ca2ff7331d1eb3194fe85ee8ab...,1726632143,20774967,314.380624,{'id': '0xc8eb2cf2f792f77af0cd9e203305a585e588...,314.500371,{'id': '0xc02aaa39b223fe8d0a0e5c4f27ead9083c75...,0xE382BBd32C4E202185762eA433278f4ED9E6151E,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,0xC8Eb2Cf2f792F77AF0Cd9e203305a585E588179D,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2
162,0xaa013ce865573532459a55487fb809c3e93edeed05af...,1726676555,20778643,144.676019,{'id': '0x6951bdc4734b9f7f3e1b74afebc670c736a0...,146.034135,{'id': '0xc02aaa39b223fe8d0a0e5c4f27ead9083c75...,0xd96E943098B2AE81155e98D7DC8BeaB34C539f01,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,0x6951bDC4734b9f7F3E1B74afeBC670c736A0EDB6,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2
405,0x1e5ddbb3a2e56692c1c261b05b482298c1cd6bd6bcf3...,1726714667,20781802,133.437132,{'id': '0x6951bdc4734b9f7f3e1b74afebc670c736a0...,134.705433,{'id': '0xc02aaa39b223fe8d0a0e5c4f27ead9083c75...,0xd96E943098B2AE81155e98D7DC8BeaB34C539f01,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,0x6951bDC4734b9f7F3E1B74afeBC670c736A0EDB6,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2
13,0xfb4645f276bf56d5ff8fef87301516ccb8ed0841a5db...,1726752383,20784928,335.615197,{'id': '0xc8eb2cf2f792f77af0cd9e203305a585e588...,335.831303,{'id': '0xc02aaa39b223fe8d0a0e5c4f27ead9083c75...,0xE382BBd32C4E202185762eA433278f4ED9E6151E,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,0xC8Eb2Cf2f792F77AF0Cd9e203305a585E588179D,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2
...,...,...,...,...,...,...,...,...,...,...,...
261,0x7356f89ae7b56795daec5ca6476ee2ce97f8124b784d...,1748255495,22566385,38.219307,{'id': '0xc8eb2cf2f792f77af0cd9e203305a585e588...,38.175659,{'id': '0x88794c65550deb6b4087b7552ecf29511379...,0xbA1462f43c6f60ebD1C62735c94E428aD073E01A,0xc4Eb861e7b66f593482a3D7E8adc314f6eEDA30B,0xC8Eb2Cf2f792F77AF0Cd9e203305a585E588179D,0x88794C65550DeB6b4087B7552eCf295113794410
5,0xfdf67dc6d3e6c48b79c91f295646fd48837d06fbbf5c...,1748287895,22569065,28.657303,{'id': '0xc8eb2cf2f792f77af0cd9e203305a585e588...,28.631744,{'id': '0x88794c65550deb6b4087b7552ecf29511379...,0xbA1462f43c6f60ebD1C62735c94E428aD073E01A,0xc4Eb861e7b66f593482a3D7E8adc314f6eEDA30B,0xC8Eb2Cf2f792F77AF0Cd9e203305a585E588179D,0x88794C65550DeB6b4087B7552eCf295113794410
127,0xbc8853882a7d05247c9fb42f8a942fdab269b60f7fe6...,1748320307,22571756,21.137299,{'id': '0x6951bdc4734b9f7f3e1b74afebc670c736a0...,21.473808,{'id': '0x88794c65550deb6b4087b7552ecf29511379...,0xe4433D00Cf48BFE0C672d9949F2cd2c008bffC04,0xc4Eb861e7b66f593482a3D7E8adc314f6eEDA30B,0x6951bDC4734b9f7F3E1B74afeBC670c736A0EDB6,0x88794C65550DeB6b4087B7552eCf295113794410
310,0x54ca0635165f7ec15b239fe39ae06e30be0932b9d9a2...,1748374523,22576246,44.264531,{'id': '0x6951bdc4734b9f7f3e1b74afebc670c736a0...,45.064088,{'id': '0xc02aaa39b223fe8d0a0e5c4f27ead9083c75...,0xe4433D00Cf48BFE0C672d9949F2cd2c008bffC04,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,0x6951bDC4734b9f7F3E1B74afeBC670c736A0EDB6,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2


In [37]:
rebalance_event_df["datetime_executed"] = pd.to_datetime(rebalance_event_df["timestamp"].astype(int), unit="s")
rebalance_event_df = rebalance_event_df.sort_values("datetime_executed")
rebalance_plan_df = rebalance_plan_df.sort_values("datetime_generated")
rebalance_event_df["day"] = rebalance_event_df["datetime_executed"].dt.date

rebalance_plan_df["day"] = rebalance_plan_df["datetime_generated"].dt.date

one_rebalance_event = rebalance_event_df.head(1)

one_rebalance_event[["datetime_executed", "destinationOutAddress", "destinationInAddress", "tokenOutAmount", "day"]]

,datetime_executed,destinationOutAddress,destinationInAddress,tokenOutAmount,day
20,2024-09-17 19:42:23,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,0xE382BBd32C4E202185762eA433278f4ED9E6151E,100.0,2024-09-17


In [ ]:
this_days_plans = rebalance_plan_df[rebalance_plan_df["day"] == one_rebalance_event["day"].values[0]].copy()

this_days_plans[
    ["datetime_generated", "destination_out", "token_out", "destination_in", "token_in", "amount_out", "day"]
]
# my destination in and out is backwards in the plans and events

,datetime_generated,destination_out,token_out,destination_in,token_in,amount_out,day
459,2024-09-17 02:16:17+00:00,0xE382BBd32C4E202185762eA433278f4ED9E6151E,0xC8Eb2Cf2f792F77AF0Cd9e203305a585E588179D,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,100.0,2024-09-17
352,2024-09-17 02:40:37+00:00,0xE382BBd32C4E202185762eA433278f4ED9E6151E,0xC8Eb2Cf2f792F77AF0Cd9e203305a585E588179D,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,100.0,2024-09-17
271,2024-09-17 02:42:56+00:00,0xE382BBd32C4E202185762eA433278f4ED9E6151E,0xC8Eb2Cf2f792F77AF0Cd9e203305a585E588179D,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,100.0,2024-09-17
345,2024-09-17 19:40:27+00:00,0xE382BBd32C4E202185762eA433278f4ED9E6151E,0xC8Eb2Cf2f792F77AF0Cd9e203305a585E588179D,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,100.0,2024-09-17
